In [ ]:
# 视频分解图片
# 1 load 2 info 3 parse 4 imshow imwrite
import cv2
import os
# 获取一个视频打开cap 参数1 file name
#cap = cv2.VideoCapture("2.mp4")
#cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) # 摄像头截取
cap = cv2.VideoCapture('./video/lift.mp4')
isOpened = cap.isOpened # 判断是否打开‘
print(isOpened)
# 获取信息 宽高
n_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('总帧数：',n_frame) # 整个视频的总帧数
fps = cap.get(cv2.CAP_PROP_FPS) # 帧率 每秒展示多少张图片
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # w
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # h
print('帧数、宽度、高度分别为：',fps,width,height) # 帧率 宽 高
i = 0 # 记录读取多少帧
frameFrequency = 24 # 每frameFrequency保存一张图片
while(isOpened):
    # 结束标志是否读取到最后一帧
    if i == n_frame:
        break
    else:
        i = i+1
    (flag,frame) = cap.read() # read方法 读取每一张 flag是否读取成功 frame 读取内容
    fileName = str(i)+'.jpg' # 名字累加
    # True表示读取成功 进行·写入
    # if 判断需要有冒号
    #if flag == True:
    outPutDirName = './image_data/' # 设置保存路径
    if not os.path.exists(outPutDirName):
        # 如果文件目录不存在则创建目录
        os.makedirs(outPutDirName)
    if i % frameFrequency == 0:
        print(fileName)
        cv2.imwrite(outPutDirName+fileName,frame,[cv2.IMWRITE_JPEG_QUALITY,100])# 质量控制 100最高
print('end!')

In [ ]:
import os
import cv2
import numpy as np 
#from skimage import compare_ssim
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import structural_similarity
#from skimage.measure import compare_ssim
'''相邻两个文件比较相似度，相似就把第二个加到新列表(删除列表）里，然后进行新列表去重，统一删除。
　　例如：有文件1-10，首先1和2相比较，若相似，则把2加入到新列表里，再接着2和3相比较，若不相似，则继续进行3和4比较...一直比到最后，然后删除新列表里的图片
   只对于连续图片（例一个视频里截下的图片）准确率也较高，其效率高'''
def delete(filename1):
    os.remove(filename1)
if __name__ == '__main__':
    path = r'./image_data/' # 文件所在路径
    img_path = path
    imgs_n = [] # 新建的删除列表
    # i = 0 # 用来统计删了第几张图片
    # img_files = [os.path.join(rootdir, file) for rootdir, _, files in os.walk(path) for file in files if
    #              (file.endswith('.jpg'))] 
    img_files=[]
    for rootdir, _, files in os.walk(path):
        files.sort(key=lambda x:int(x[:-4]))
    for file in files:
         if file.endswith('.jpg'):
            img_files.append(os.path.join(rootdir, file))
   
    for currIndex, filename in enumerate(img_files):
        if not os.path.exists(img_files[currIndex]):
            print('not exist', img_files[currIndex])
            break
        img = cv2.imread(img_files[currIndex])
        img1 = cv2.imread(img_files[currIndex + 1])
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        # https://cloud.tencent.com/developer/section/1414961 开发者手册链接
        # 本程序最核心的代码
        # skimage.measure.compare_ssim（X，Y，...）
        # 计算两幅图像之间的平均结构相似性指数
        #ssim = ssim(img, img1,multichannel=True,win_size=7)
        ssim=structural_similarity(img, img1,channel_axis=-1)
        # 相似度大于0.9就加入删除列表
        if ssim > 0.93:
            imgs_n.append(img_files[currIndex + 1]) # 加入新建删除列表
            i = i+1 # 每加入一个就会删除
            print(img_files[currIndex], img_files[currIndex + 1], ssim)
            print('删除次数', i)
        # 否则不删除，给出两图片的相似度
        else:
            print('small_ssim',img_files[currIndex], img_files[currIndex + 1], ssim)
        currIndex += 1
        if currIndex >= len(img_files)-1:
            break
    for image in imgs_n:
        delete(image)